In [1]:
print('get qualified txs')
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import flipside_utils as f
import clickhouse_utils as ch
import csv_utils as cu
import google_bq_utils as bqu
sys.path.pop()

import time

import numpy as np
import pandas as pd
import os
import clickhouse_connect as cc

get qualified txs


In [2]:
ch_client = ch.connect_to_clickhouse_db() #Default is OPLabs DB

query_name = 'daily_evms_qualified_txs_counts'

In [3]:
col_list = [
        'dt','blockchain','name','layer','chain_id'
        , 'num_raw_txs', 'num_success_txs','num_qualified_txs','source'
        ]

In [4]:
flipside_configs = [
        {'blockchain': 'blast', 'chain_id': 238, 'name': 'Blast', 'layer': 'L2', 'trailing_days': 365}
]
clickhouse_configs = [
        {'blockchain': 'metal', 'chain_id': 1750,'name': 'Metal', 'layer': 'L2', 'trailing_days': 365},
        {'blockchain': 'mode', 'chain_id': 34443,'name': 'Mode', 'layer': 'L2', 'trailing_days': 365},
        {'blockchain': 'bob', 'chain_id': 60808,'name': 'BOB (Build on Bitcoin)', 'layer': 'L2', 'trailing_days': 365},
        {'blockchain': 'fraxtal', 'chain_id': 252,'name': 'Fraxtal', 'layer': 'L2', 'trailing_days': 365},
]

In [5]:
# Run Flipside
print('     flipside runs')
flip_dfs = []
with open(os.path.join("inputs/sql/flipside_bychain.sql"), "r") as file:
                        og_query = file.read()

for chain in flipside_configs:
        print(     'flipside: ' + chain['blockchain'])
        query = og_query
        #Pass in Params to the query
        query = query.replace("@blockchain@", chain['blockchain'])
        query = query.replace("@chain_id@", str(chain['chain_id']))
        query = query.replace("@name@", chain['name'])
        query = query.replace("@layer@", chain['layer'])
        query = query.replace("@trailing_days@", str(chain['trailing_days']))
        
        df = f.query_to_df(query)

        flip_dfs.append(df)

flip = pd.concat(flip_dfs)
flip['source'] = 'flipside'
flip['dt'] = pd.to_datetime(flip['dt']).dt.tz_localize(None)
flip = flip[col_list]


     flipside runs
flipside: blast


In [6]:
# Run Dune
print('     dune runs')
dune_df = d.get_dune_data(query_id = 3740822, #https://dune.com/queries/3740822
    name = "dune_evms_qualified_txs",
    path = "outputs",
    performance="large"
)
dune_df['source'] = 'dune'
dune_df['dt'] = pd.to_datetime(dune_df['dt']).dt.tz_localize(None)
dune_df = dune_df[col_list]

2024-06-11 16:22:56.173 | INFO     | duneapi_utils:get_dune_data:59 - Results available at https://dune.com/queries/3740822


     dune runs


2024-06-11 16:22:57.262 | INFO     | duneapi_utils:get_dune_data:106 - ✨ Results saved as outputs/dune_evms_qualified_txs.csv, with 4621 rows and 9 columns.


In [7]:
# Run Clickhouse
print('     clickhouse runs')
ch_dfs = []
with open(os.path.join("inputs/sql/goldsky_bychain.sql"), "r") as file:
                        og_query = file.read()

for chain in clickhouse_configs:
        print(     'clickhouse: ' + chain['blockchain'])
        query = og_query
        #Pass in Params to the query
        query = query.replace("@blockchain@", chain['blockchain'])
        query = query.replace("@chain_id@", str(chain['chain_id']))
        query = query.replace("@name@", chain['name'])
        query = query.replace("@layer@", chain['layer'])
        query = query.replace("@trailing_days@", str(chain['trailing_days']))
        
        df = ch_client.query_df(query)

        ch_dfs.append(df)

ch = pd.concat(ch_dfs)
ch['source'] = 'goldsky'
ch['dt'] = pd.to_datetime(ch['dt']).dt.tz_localize(None)
ch = ch[col_list]

     clickhouse runs
clickhouse: metal
clickhouse: mode
clickhouse: bob
clickhouse: fraxtal


In [8]:
# Step 1: Filter dune_df for chains not in flip
filtered_dune_df = dune_df[~dune_df['chain_id'].isin(flip['chain_id'])]
# Step 2: Union flip and filtered_dune_df
combined_flip_dune = pd.concat([flip, filtered_dune_df])
# Step 3: Filter ch for chains not in combined_flip_dune
filtered_ch = ch[~ch['chain_id'].isin(combined_flip_dune['chain_id'])]
# Step 4: Union the result with filtered_ch
final_df = pd.concat([combined_flip_dune, filtered_ch])
# final_df


In [9]:
opstack_metadata = pd.read_csv('../op_chains_tracking/outputs/chain_metadata.csv')

opstack_metadata['chain_id'] = opstack_metadata['mainnet_chain_id']

meta_cols = ['is_op_chain','op_based_version', 'alignment','chain_name', 'display_name']

final_enriched_df = final_df.merge(opstack_metadata[meta_cols], on='chain_id', how = 'left')
final_enriched_df['alignment'] = final_enriched_df['alignment'].fillna('Other EVMs')
final_enriched_df['is_op_chain'] = final_enriched_df['is_op_chain'].fillna(False)
final_enriched_df['display_name'] = final_enriched_df['display_name'].fillna(final_enriched_df['name'])

final_enriched_df = final_enriched_df.drop(columns=['name'])

KeyError: "['display_name_lower'] not in index"

In [ ]:
final_enriched_df.sort_values(by=['dt','blockchain'], ascending =[False, False], inplace = True)

final_enriched_df.to_csv('outputs/'+query_name+'.csv', index=False)

In [ ]:
#BQ Upload
bqu.write_df_to_bq_table(final_enriched_df, query_name)